# importations 

In [1]:
import numpy as np # linear algebra
import pandas as pd 
import os
import optuna
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
from tqdm import tqdm
tqdm.pandas()
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Explore the data

In [ ]:
#pip install ydata-profiling

In [2]:
sample = pd.read_csv("/kaggle/input/playground-series-s4e6/sample_submission.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")
train = pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
blend = pd.read_csv("/kaggle/input/blendfromsub/6_Blend.csv")
testo = pd.read_csv("/kaggle/input/original/Test.csv")
data = pd.read_csv("/kaggle/input/original/data.csv")

The data dataframe is not well organized and I didn't take the time to organize it so I didn't use it 
You should try experimenting with it, it increases the score

In [4]:
y = train.pop("Target")
x = train

In [5]:
#profile = ProfileReport(x,title = "Report")
#profile.to_notebook_iframe()

# Feature Engineering and normalization

In [ ]:
MMscaler = MinMaxScaler()
Stdscaler = StandardScaler()
#Min Max Scaling
def prepro(df):    
    df['Course_Normalize']=(df['Course']-df['Course'].min())/(df['Course'].max()-df['Course'].min())
    transformed_data, best_lambda = stats.boxcox(df['Nacionality'])
    df['Nacionality_transf'] = transformed_data
    df['MotherQual_Normalize']=(df["Mother's qualification"]-df["Mother's qualification"].min())/(df["Mother's qualification"].max()-df["Mother's qualification"].min())

    df["Previous qualification (grade) normalize"]=(df["Previous qualification (grade)"]-df["Previous qualification (grade)"].min())/(df["Previous qualification (grade)"].max()-df["Previous qualification (grade)"].min())
    df["Application mode tran"], best_lambda = stats.boxcox(df['Application mode'])

    df["Age at enrollment Normalize"] = MMscaler.fit_transform(df["Age at enrollment"].values.reshape(-1,1))
    df["Course tran"], best_lambda = stats.boxcox(df['Course'])
prepro(x)
prepro(test)

# Modeling

In [ ]:
numerical_cols = x.select_dtypes(exclude=['object']).columns.tolist()
categorical_cols = x.select_dtypes('object').columns.tolist()
print(f"Numerical: {numerical_cols}, \nCategorical: {categorical_cols}")

# Feature Encoding for the target

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y)

In [ ]:
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

models = [
    #RandomForestClassifier(n_estimators= 200, min_samples_split=5, min_samples_leaf= 2, max_depth= None),
    #XGBClassifier(n_estimators =  468, max_depth =  7, learning_rate = 0.03914830481081083, subsample =  0.7639287344761662, colsample_bytree = 0.8932105363776578, min_child_weight = 4, gamma =  0.02497911090390904, reg_alpha= 0.1535239781973239, reg_lambda= 0.0014053600399553418),
    LGBMClassifier(subsample =  0.8, num_leaves= 50, n_estimators= 500, min_child_samples= 100, max_depth= 20, learning_rate=0.01,colsample_bytree= 0.8),
    CatBoostClassifier(iterations= 463,depth = 7, learning_rate=0.07575039054751037, l2_leaf_reg = 0.14582515638980897, bagging_temperature = 0.1497808176808535, border_count= 97, random_strength =  0.002688504419743385,silent = True)
]

for model in models:
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    score = cross_val_score(pipeline, x, y_train, cv=5)
    print(f"{type(model).__name__}: {score.mean()}")


# Optimization using Optuna

In [ ]:
"""from sklearn.model_selection import StratifiedKFold
# Assuming numerical_cols and categorical_cols are defined
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])

# Define the objective function for Optuna
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 500),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10, log=True),
        'random_state': 42
    }
    
    model = CatBoostClassifier(**params, silent=True)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # StratifiedKFold ensures the split preserves the percentage of samples for each class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    score = cross_val_score(pipeline, x, y_train, cv=skf, n_jobs=-1, scoring='accuracy')
    return score.mean()

# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Best Parameters:", study.best_params)
print("Best CV Score:", study.best_value)
# Extract the best model"""

In [ ]:
 best_pipeline = Pipeline([
     ('preprocessor', preprocessor),
     ('model', models[0])
 ])
 best_pipeline.fit(x, y)


submission = pd.DataFrame({ 
    "id" : test["id"], 
    "Target" : best_pipeline.predict(test)
})
submission.to_csv("submissionLGBM1.csv", index=False)

# Submission

In [ ]:
best_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', models[1])
])
best_pipeline.fit(x, y)

# Ensure predictions are 1-dimensional
predictions = best_pipeline.predict(test).flatten()

submission = pd.DataFrame({
    "id": test["id"], 
    "Target": predictions
})
submission.to_csv("submissionCATBOOST.csv", index=False)

In [ ]:
df = pd.read_csv('/kaggle/working/submissionXGBC.csv')
# Define the mapping from numeric values to string labels
mapping = {0: 'Dropout', 2: 'Graduated', 1: 'Enrolled'}

# Apply the mapping to the 'target' column
df['Target'] = df['Target'].map(mapping)

# Save the modified DataFrame back to a CSV file
output_file_path = 'modified_submission.csv'  # Replace with your desired output file path
df.to_csv(output_file_path, index=False)

# Try ensemble voting

In [ ]:
import pandas as pd
from scipy.stats import mode

# Load the submission files
submission1 = pd.read_csv('/kaggle/working/submissionCATBOOST.csv')
submission2 = pd.read_csv('/kaggle/working/submissionLGBM1.csv')
# Ensure they are sorted by the same ID if needed
submission1 = submission1.sort_values('id')
submission2 = submission2.sort_values('id')
# Combine the predictions into a DataFrame
combined_predictions = pd.DataFrame({
    'id': submission1['id'],
    'Target1': submission1['Target'],
    'Target2': submission2['Target'],
})

# Apply majority voting
combined_predictions['FinalPrediction'] = combined_predictions[['Target1', 'Target2']].mode(axis=1)[0]

# Save the final submission file
final_submission = combined_predictions[['id', 'FinalPrediction']]
final_submission.columns = ['id', 'Target']  # Ensure column names match the required submission format
final_submission.to_csv('final.csv', index=False)
